---
<big><big><big><big><big><big>Variational autoencoder</big></big></big></big></big></big>

---
<img style="float: right;" src="gmum.png" width="25%">

---

<id=tocheading><big><big><big><big>Sections</big></big></big></big>
<div id="toc"></div>

---

In [42]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Generative vs. discriminative models in ML
<img style="float: right;" src="discriminative_vs_generative.png" width="50%">
1. __discriminative__ compute _target_ values __only__ for __observable__ variables $p(c\mid x)$
  * e.g. logistic regression, SVM, maximum entropy, neural networks
2. __generative__ find target __both__ for observable and _hidden_ variables
  * hidden are somehow computed from observable
  * model density distributions and __then__ conditional densities
  * are able to generate __new__ data
  * e.g. Gaussian mixture, Hidden Markov, probabilistic grammars, Naive Bayes, Latent Dirichlet, Restricted Boltzman, Generative Adversarial Networks GAN, VAE

## Need of generetive models
they say that Richard Feynman said "_What I cannot create, I do not understand_"
<img style="float: right;" src="generative_models.png" width="75%">
1. we have lots of data
2. how to generate new data?
  * draw a __code__
  * train a model which would generate a __valid__ example of that code
  * e.g. images, text, strings describing new _active) chemical molecules,...
3. possible approaches
  * __Generative Adversarial Networks GANs__
    1. draw a _true_ image or a _generated_ one
    2. __discriminator__ $Dis$ decide whether true or generated maximizing/minimizing 
    $$L_{GAN}=\log(Dis(x))+\log(1-Dis(Gen(z)))$$
  * __Pixel RNN__
    * autoregressive models
    * conditional distributions of elements (pixels) on neighbouring ones
  * __Variational AutoEncoders VAE__
4. Generative models __should__ be able to learn features needed for classification

# Variational autoencoders VAE

## Model
<img style="float: right;" src="vae_graphical_model.png" width="35%">

$N$ examples $X=\{x^{(i)}\}_{i=1}^N$ generated by process
  * $z^{(i)}$ generated from _prior_ distribution $p_{\theta^\ast}(z)$
  * $x^{(i)}$ from _conditional_ distr. $p_{\theta^\ast}(x\mid z)$


1. true $\theta^\ast$ is unknown
2. $z$ values are unknown 


Problems
1. marginal likelihood
$$p_\theta(x)=\int p_\theta(z)\,p_\theta(x\mid z)\,dz$$ is _intractable_
  * intractable posterior $$p_\theta(z\mid x)=\frac{p_\theta(x\mid z)\,p_\theta(z)}{p_\theta(x)}$$
  and Expectation-Maximization EM algorithm cannot be used
2. $N$ is large 
  * paramter updates need to be done on small mini-batches
  * Monte Carlo MCMC would be too slow

## Interested in
<img style="float: right;" src="vae_graphical_model.png" width="35%">
1. __maximum likelihood MLE__ or __maximum a posteriori MAP__ _estimation_ of $\theta$
2. inference of _posterior_ $z$ given $x$
  * efficient!
  * useful for coding and/or representation tasks
  * e.g. representation of discrete values in __continuous__ _latent space_
3. inference of $x$
  * image denoising, inpainting, resolution enhancement
4. distribution of latent space
  * rather in distribution of $p_\theta(z\mid x)$
  * possible __traversal__ (__exploration__) of latent space

## Solutions
<img style="float: right;" src="vae_graphical_model.png" width="35%">
1. definition of model $q_\phi(z\mid x)$
  * approximation of _intractable_ $p_\theta(z\mid x)$
  * parameters $\theta$ and $\phi$ trained jointly
2. the whole model composed of
  * $q_\phi(z\mid x)$ a probabilistic __encoder__
    * given $x$ produces a __distribution__ over latent space!
    * __posterior__ net
  * $p_\theta(x\mid z)$ a probabilistic __decoder__
    * given $z$ produces a __distribution__ of related $x$ values
    * __likelihood__ net
3. generating __new__ data from latent consists of 
  * __sampling__ $z$ latent space distribution 
  * then sampling from $p_\theta(x\mid z)$
  * $q_\phi(z\mid x)$ and $p_\theta(z\mid x)$ should be identical, but 
  $$p_\theta(z\mid x)=\frac{p_\theta(x\mid z)\,p_\theta(z)}{p_\theta(x)}$$ is intractable
4. solution: use __Kullback-Leibler__ divergence 
$$D_{KL}(p\,\rVert\,q)=\int_Xp(x)\ln\frac{p(x)}{q(x)}=\int_Xp(x)\ln\,p(x)-p(x)\ln\,q(x)$$
  * asymmetric, __not__ a distance, but
    * set $p$
    * ask whether $q$ approaches $p$
    and approximate
    $$D_{KL}(p\,\rVert\,q)=\int_X\underbrace{p(x)\ln\,p(x)}_{const}-\underbrace{p(x)}_{set}\ln\,q(x)=const-\ln\,q(x)$$
5. Variational Autoencoder cost function
$$L_{VAE}=D_{KL}(p_{DATA}\,\rVert\,p_\theta(x))+\int_{x\sim\,p_{DATA}}D_{KL}(q_\phi(z\mid\,x)\,\rVert\,p_\theta(z\mid\,x)),$$
where 
  * $D_{KL}(p_{DATA}\,\rVert\,p_\theta(x))$ is the __reconstruction cost__
  * $\int_{x\sim\,p_{DATA}}D_{KL}(q_\phi(z\mid\,x)\,\rVert\,p_\theta(z\mid\,x))$ is the __prior distribution cost__
  * the latent space distribution $p(z)$ is usually $\mathcal{N}(0, \mathrm{I})$

## VAE architecture
<img src="vae-architecture.png" width="80%">
1. posterior network $q_\phi$
  * computes the __parameters__ for the sampling from the latent space $z\sim\mathcal{N}(0,\mathrm{I})$
2. latent space
  * sampling from the latent space
3. likelihood network $p_\theta$
  * sampling from output (also $z\sim\mathcal{N}(0,\mathrm{I})$)
4. loss function
  * $D_{KL}$ of the prior distribution vs $q_\phi$
  * recall loss as a negative likelihood -- cross entropy

In [22]:
import tensorflow as tf
from tensorflow.contrib.distributions import kl_divergence
from tensorflow.contrib.distributions import MultivariateNormalDiag
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from time import time

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
inp_dim = 784
x = tf.placeholder(dtype=tf.float32, shape=[None, inp_dim], name="x")

In [7]:
# encoder's layer size
enc_dim = 100
enc_w = tf.get_variable("enc_w", shape=[inp_dim, enc_dim], dtype=tf.float32,
                       initializer=tf.contrib.layers.xavier_initializer())
enc_b = tf.get_variable("enc_b", shape=[enc_dim], dtype=tf.float32,
                       initializer=tf.contrib.layers.xavier_initializer())
enc_hid = tf.nn.relu(tf.matmul(x, enc_w) + enc_b)

In [8]:
# latent layer size
z_dim = 200
# mu and log sigma networks
z_mu_w = tf.get_variable("z_mu_w", shape=[enc_dim, z_dim], dtype=tf.float32, 
                         initializer=tf.contrib.layers.xavier_initializer())
z_mu_b = tf.get_variable("z_mu_b", shape=[z_dim], dtype=tf.float32,
                         initializer=tf.contrib.layers.xavier_initializer())
z_mu = tf.matmul(enc_hid, z_mu_w) + z_mu_b

z_log_sigma_w = tf.get_variable("z_log_sigma_w", shape=[enc_dim, z_dim], 
                                dtype=tf.float32, 
                                initializer=tf.contrib.layers.xavier_initializer())
z_log_sigma_b = tf.get_variable("z_log_sigma_b", shape=[z_dim], dtype=tf.float32,
                                initializer=tf.contrib.layers.xavier_initializer())
# clip log(sigma)
z_log_sigma = tf.maximum(tf.matmul(enc_hid, z_log_sigma_w) + z_log_sigma_b, -3.0)

In [9]:
# sample from latent space z
mvn = MultivariateNormalDiag(loc=z_mu, scale_diag=tf.exp(z_log_sigma), name="mvn")
z_sample = mvn.sample(name="z_sample")

In [10]:
# decoder layer size
dec_dim = 100
dec_w = tf.get_variable("dec_w", shape=[z_dim, dec_dim], dtype=tf.float32,
                       initializer=tf.contrib.layers.xavier_initializer())
dec_b = tf.get_variable("dec_b", shape=[dec_dim], dtype=tf.float32,
                       initializer=tf.contrib.layers.xavier_initializer())
dec_hid = tf.nn.sigmoid(tf.nn.relu(tf.matmul(z_sample, dec_w) + dec_b))

In [11]:
# mu and log sigma for decoder sampling
dec_mu_w = tf.get_variable("dec_mu_w", shape=[dec_dim, inp_dim], dtype=tf.float32, 
                           initializer=tf.contrib.layers.xavier_initializer())
dec_mu_b = tf.get_variable("dec_mu_b", shape=[inp_dim], dtype=tf.float32,
                         initializer=tf.contrib.layers.xavier_initializer())
dec_mu = tf.matmul(dec_hid, dec_mu_w) + dec_mu_b

dec_log_sigma_w = tf.get_variable("dec_log_sigma_w", shape=[dec_dim, inp_dim], 
                                  dtype=tf.float32, 
                                  initializer=tf.contrib.layers.xavier_initializer())
dec_log_sigma_b = tf.get_variable("dec_log_sigma_b", shape=[inp_dim], 
                                  dtype=tf.float32,
                                  initializer=tf.contrib.layers.xavier_initializer())
# clip log(sigma)
dec_log_sigma = tf.maximum(
                    tf.matmul(dec_hid, dec_log_sigma_w) + dec_log_sigma_b, -3.)

In [12]:
# sample log_prob from output
p_theta_x_dist = MultivariateNormalDiag(loc=dec_mu, 
                                        scale_diag=tf.exp(dec_log_sigma), 
                                        name="p_theta_x_dist")
p_theta_x = tf.reduce_mean(p_theta_x_dist.log_prob(x))

In [13]:
# Kullback-Leibler divergence
post_distribution = mvn
prior_distribution = MultivariateNormalDiag(tf.zeros(z_dim),
                                            scale_diag=tf.ones(z_dim))
kl_loss = tf.divide(
    tf.reduce_mean(kl_divergence(post_distribution, prior_distribution)),
    float(z_dim))

In [14]:
# negative cross entropy
cross_ent = -p_theta_x
# cross_ent = -tf.reduce_mean(-tf.reduce_sum(x * tf.log(-p_theta_x), 
#                                          reduction_indices=[1]))
cross_ent = tf.divide(tf.reduce_mean(cross_ent), float(inp_dim))

# loss function
# loss_func = inp_dim * kl_loss + z_dim * cross_ent
loss_func = z_dim * kl_loss + inp_dim * cross_ent

# optimizer
# optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001).minimize(loss_func)
optimizer = tf.train.AdamOptimizer().minimize(loss_func)

In [37]:
session = tf.Session()
glob_init = tf.global_variables_initializer()
loops = 125000
batch_size = 512
interval = 2500

session.run(glob_init)
s = time()
for k in range(loops):
    data, _ = mnist.train.next_batch(batch_size=batch_size)
    _, ls_fn, cr_ent, kl_ls = session.run([optimizer, loss_func, cross_ent, kl_loss],
                                          feed_dict = {x: data})
    if k == 0 or (k + 1) % interval == 0:
        t = time()
        print("{:6d}: {:+11.5f} < (cr {:+8.5f}, D_KL {:+8.5f})\t[{:.0f}s]".format(
            k + 1, ls_fn, cr_ent, kl_ls, (t - s)))
    s = t

     1:  +828.22321 < (cr +1.03229, D_KL +0.09455)	[0s]
  2500:  -919.36005 < (cr -1.19686, D_KL +0.09488)	[235s]
  5000:  -992.02509 < (cr -1.29638, D_KL +0.12169)	[233s]
  7500: -1029.88867 < (cr -1.34847, D_KL +0.13655)	[223s]
 10000: -1056.85059 < (cr -1.38515, D_KL +0.14553)	[221s]
 12500: -1058.08643 < (cr -1.38956, D_KL +0.15665)	[222s]
 15000: -1072.38159 < (cr -1.40908, D_KL +0.16168)	[221s]
 17500: -1077.74231 < (cr -1.41716, D_KL +0.16655)	[222s]
 20000: -1090.44849 < (cr -1.43468, D_KL +0.17170)	[222s]
 22500: -1083.87842 < (cr -1.42722, D_KL +0.17532)	[221s]
 25000: -1091.44714 < (cr -1.43739, D_KL +0.17734)	[222s]
 27500: -1108.40796 < (cr -1.45946, D_KL +0.17904)	[222s]
 30000: -1108.21216 < (cr -1.45919, D_KL +0.17897)	[227s]
 32500: -1106.80237 < (cr -1.45797, D_KL +0.18122)	[221s]
 35000: -1109.13440 < (cr -1.46135, D_KL +0.18280)	[223s]
 37500: -1111.41016 < (cr -1.46453, D_KL +0.18390)	[222s]
 40000: -1112.04785 < (cr -1.46570, D_KL +0.18530)	[222s]
 42500: -1105.60

In [29]:
from PIL import Image

In [30]:
def draw_grid(arr):
    return np.concatenate([np.concatenate(row, axis=1) for row in arr], axis=0)

In [43]:
batch_size = 10
digit_shape = (28, 28)
digits, _ = mnist.train.next_batch(batch_size=batch_size)
inp, out = session.run([x, dec_mu], feed_dict={x: digits})
arr = np.stack((inp, out)) * 255
arr = arr.reshape(arr.shape[:2] + digit_shape)
img = Image.fromarray(draw_grid(arr))
img.show()

In [35]:
# linear interpolation
steps = 14
#digits, _ = mnist.train.next_batch(batch_size=batch_size)
z_values = session.run([z_sample], feed_dict={x: digits})[0]
zzz = np.array([]).reshape((0, z_values.shape[-1]))
for k in range(batch_size - 1):
    z_from = z_values[k]
    z_to = z_values[k + 1]
    this_zzz = np.stack([p * z_from + (1 - p) * z_to 
                         for p in np.linspace(1, 0, steps)])
    zzz = np.vstack((zzz, this_zzz))
arr = session.run([dec_mu], feed_dict={z_sample: zzz})[0]
arr[arr < 0] = 0
arr[arr > 1] = 1
arr = arr.reshape((-1, steps) + digit_shape)
img = Image.fromarray(draw_grid(255 * arr))
img.show()

In [45]:
# more polar interpolation
steps = 14
#digits, _ = mnist.train.next_batch(batch_size=batch_size)
z_values = session.run([z_sample], feed_dict={x: digits})[0]
zzz = np.array([]).reshape((0, z_values.shape[-1]))
for k in range(batch_size - 1):
    z_from = z_values[k]
    z_to = z_values[k + 1]
    this_zzz = np.stack([np.sqrt(p) * z_from + np.sqrt((1 - p)) * z_to 
                        for p in np.linspace(1, 0, steps)])
#    this_zzz = np.stack([(np.cos(p * np.pi / 2) * z_from + np.sin(p * np.pi / 2) * z_to) 
#                         for p in np.linspace(0, 1, steps)])
    zzz = np.vstack((zzz, this_zzz))
arr = session.run([dec_mu], feed_dict={z_sample: zzz})[0]
arr[arr < 0] = 0
arr[arr > 1] = 1
arr = arr.reshape((-1, steps) + digit_shape)
img = Image.fromarray(255 * draw_grid(arr))
img.show()

# Applications?
1. generation of new data
2. learning of features from an extended data set
3. generation of data fulfilling __with__ some attributes
4. organization of the feature space

## VAE in search of automata
<img style="float: right;" src="automaton.png" width="45%">
__Synchronizing automaton__ if there exists a word that is to leave the automaton in one particular _whatever_ state it started at.

__Cerny conjecture:__ the shortest reset word for a synchronizing automaton has a length at most $(n-1)^2$ (1964)
  * finding the shortest reset word is an NP problem
  * still an open problem!
  * with number of states $n$ growing the probability of an automaton being synchronizable approaches $1$
  * it is very hard to find an __extreme__ automaton, i.e. one with reset word of $(n-1)^2$ length
  * only a few except are known for larger $n$
  
Is it possible to find some extreme or almost-extreme automata using ML methods?

### VAE architecture used
<img src="vae-for-automata.png" width="100%">

1. 1-D convolution layers in the posterior (encoder) network
  * automata represented serialized
    * with higher number of states and larger alphabet it may be hard
2. latent space with $\mathcal{N}(0,\mathrm{I})$ distribution
  * last layer of the posterior network has 2 linear fully connected networks
    * approximation of $\mu$ mean value of $z$
    * approximation of $\log\,\sigma$
  * $z$ is __sampled__ from the given distribution
3. likelihood network
  * a series of recurrent networks (GRU here, LSTM's might be used as well)
  * input word is set as __repeated__ value of $z$
  * values are processed in sequence modifying the memory state
4. output value is decoded

### Decoding and interpolation
<img src="linear-interpolation-automata-1.png" width="110%">
1. interpolation through the latent space
  * take two automata
  * find (sample) their $z$ images
  * interpolate linearly between them
  * some results (or after small predefined modifications) represent valid automata
  * several new data __do not__ come from training set -- a generative model!
2. But...
<img style="float: right;" src="latent-distance-from-mean.png" width="65%">
  * histogram of distances of points sampled from $\mathcal{N}(0,\mathrm{I})$ latent space
  * the distance from the center of $$z=X_1^2+\dots+X_d^2$$ follows a $\chi^2(d)$ distribution
    * mean value $\mathbf{E}\|z\|^2=d$
    * variance $Var\left[\frac{\|z\|^2}{d}\right]=\frac{2}{d}$
    * $\chi^2(d)$ is a special type of _gamma_ distribution $\Gamma(\frac{d}{2}, 2)$
    * for $d>30$ it is very similar to $\mathcal{N}(\sqrt{2d-1}, 1)$
  * the high dimensional $\mathcal{N}(0,\mathrm{I})$ distribution is actually a sphere - it _looks_ like a __soap bubble__
    * in high dimensions most data in a ball are actually in a very thin skin of it
    * true not only for gaussian distribution
    * two randomly chosen $z_1$ and $z_2$ have virtually the same norm
    * linear interpolation $a*z_1+(1-a)*z_2$ has norm $\sqrt{a^2+(1+a)^2}<1$
  * linear interpolation necessesarily passes through an empty space...
  * better a polar coordinates interpolation $$\sqrt{a}\,z_1+\sqrt{1-a}\,z_2$$
  going through the sphere
3. an encode-decode interpolation
  * start with linear
  * decode strings __and__ encode them again $k$ times
  
<img src="linear-interpolation-automata-2.png" width="110%">
  * more results represent valid automata
  * there are more valid automata not from the training set
  
4. other possibilieties
  * change the prior distribution

### Looking for a manifold
1. a well known phenomenon is that when a problem is given in $K$ dimensional space, it is generally placed on a manifold of a __much lower dimension__
  * several dimensions are actually spurious
    * dimensionality reduction methods
  * __manifold learning__ adresses this by trying to identify this mamifold and learn __there__
    * e.g. swiss-roll structure
2. problem: find a manifold where the data are placed so that it would be possible to perform simple __arithmetical__ operations which modify attributes

<img src="automata-manifold.png" width="110%">

# Problems
1. VAEs return _fuzzy_ images (and data)
  * problem of distribution $\mathcal{N}(\mu,\sigma)$ returned
  * vary small modifications of the prior space
  * cooperation with GANs
  * other distributions of the prior space
    * but the bubble effect is true not only for the Gaussian

# Some literature
1. Kingma, Welling, _Auto encoding variational Bayes_, arXiV:1312.6114, stat.ML, 2014, __basic paper describing the training of the model__
2. Gomez-Bombarelli, Duvenaud _et al_, _Automatic chemical design using a data-driven continuous representation of molecules_, arXiV:1610.02415, cs.LG, 2015, __use of VAE in modelling drug-like discrete small molecules for the search of new active ones__
3. Larsen, Sonderby, Larochelle, Winther, _Autoencoding beyond pixels using a learned similarity metric_, arXiV:1512.09300, cs.LG, February 2016, __a composition of VAE together with a GAN; exploration of latent space with attributes__
4. Rezende, Mohamed, _Variational inference with normalizing flows", arXiV:1505.05770, stat.ML, 2016, __modification of posterior distributions with a series of invertible transformations__